In [1]:
import tensorflow as tf 
import random
from cv2 import GaussianBlur
import numpy as np
from tensorflow.keras import datasets, layers, models

def getDataset():
  (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
  normalize = lambda x: x/x.max()
  x_train = normalize(x_train)
  x_test = normalize(x_test)
  createBlurred = lambda arr: np.array([GaussianBlur(x, (5,5), ((random.random()*3)+1)) + (0.02*np.random.normal(0,1, (32, 32, 3))) for x in arr])
  return (createBlurred(x_train), x_train), (createBlurred(x_test), x_test)

In [2]:
(x_train, y_train), (x_test, y_test) = getDataset()

170498071/170498071 [==============================] - 13s 0us/step


In [3]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout
from tensorflow.keras.models import Model

def get_CAE(input_shape):
    inp = Input(input_shape)
    x = inp
    x = Conv2D(filters=32, kernel_size=3, strides=1, activation="relu", padding="same")(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Conv2D(filters=128, kernel_size=3, strides=1, activation="relu",  padding="same")(x)
    x = Conv2D(filters=256, kernel_size=3, strides=1, activation="relu",  padding="same")(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Conv2D(filters=256, kernel_size=3, strides=1, activation="relu",  padding="same")(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Conv2DTranspose(filters=256, kernel_size=3, strides=2, activation="relu", padding="same")(x)
    x = Conv2DTranspose(filters=128, kernel_size=3, strides=2, activation="relu", padding="same")(x)
    x = Conv2DTranspose(filters=32, kernel_size=3, strides=2, activation="relu", padding="same")(x)
    out = Conv2D(filters=3, kernel_size=3, strides=1, activation="relu", padding="same")(x)

    model = Model(inp,out)

    print(model.summary())
    return model

input_shape = x_train.shape[1:]

cae_model = get_CAE(input_shape)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       36992     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                         

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

cae_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


callbacks = [EarlyStopping(monitor="val_loss", patience=3)]
history_cae = cae_model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test,y_test), callbacks=callbacks)

Epoch 1/20
1563/1563 [==============================] - 32s 12ms/step - loss: 0.0111 - accuracy: 0.6844 - val_loss: 0.0053 - val_accuracy: 0.7555
Epoch 2/20
1563/1563 [==============================] - 19s 12ms/step - loss: 0.0050 - accuracy: 0.7896 - val_loss: 0.0047 - val_accuracy: 0.7921
Epoch 3/20
1563/1563 [==============================] - 20s 13ms/step - loss: 0.0045 - accuracy: 0.8023 - val_loss: 0.0042 - val_accuracy: 0.8174
Epoch 4/20
1563/1563 [==============================] - 20s 13ms/step - loss: 0.0041 - accuracy: 0.8113 - val_loss: 0.0039 - val_accuracy: 0.8246
Epoch 5/20
1563/1563 [==============================] - 19s 12ms/step - loss: 0.0039 - accuracy: 0.8169 - val_loss: 0.0038 - val_accuracy: 0.8347
Epoch 6/20
1563/1563 [==============================] - 19s 12ms/step - loss: 0.0037 - accuracy: 0.8201 - val_loss: 0.0036 - val_accuracy: 0.7868
Epoch 7/20
1563/1563 [==============================] - 20s 13ms/step - loss: 0.0036 - accuracy: 0.8224 - val_loss: 0.0039 -

In [ ]:
def plot_history(model_history,keys):
    m,val_m = keys
    plt.plot(model_history.history[m])
    plt.plot(model_history.history[val_m])
    plt.ylabel(m)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

plot_history(history_cae,['accuracy','val_accuracy'])
plot_history(history_cae,['loss','val_loss'])

In [ ]:
predictions = cae_model.predict(x_test)